In [1]:
# I could not install Spark on local PyCharm application. 
# That's why I am using Colab as you already suggested in the lecture.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install specific Java and Spark for Python.
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15

In [3]:
# Check to see if I installed Spark correctly or not.
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [4]:
from pyspark import SparkContext
from itertools import combinations

sc = SparkContext.getOrCreate()
rdd = sc.textFile("/content/drive/MyDrive/Colab Notebooks/BDP/Datasets-20210410/Lottery.txt")  #1995 - 2020

rdd = rdd.flatMap(lambda line : line.split('"rakamlar":'))
rdd = rdd.map(lambda line : line[2:19])
rdd = rdd.filter(lambda x : 'bilenKis' not in x)

def split_and_combin(line):
  lin = line.split("#")
  num_list = [int(lin[0]), int(lin[1]), int(lin[2]), int(lin[3]), int(lin[4]), int(lin[5])]
  # I examined the dataset and saw that the lottery numbers seem already sorted. But let's sort it for guarantee.
  num_list.sort()
  # itertools.combinations(iterable, r) has the followin property: If the input iterable is sorted, the combination tuples will be produced in sorted order.
  # So there is no need for sorting for later operations since the input (num_list) is already sorted
  comb_list = list(combinations(num_list, 3))
  return comb_list

rdd = rdd.flatMap(lambda x: split_and_combin(x))
rdd = rdd.map(lambda x: (x, 1)) # indexing trick
rdd = rdd.reduceByKey(lambda x, y: x+y) # reduce operation over the keys
rdd = rdd.sortBy(lambda x: x[1], ascending=False) # sort according to possibility in descending
print("Top 10 most commonly drawn triple numbers (not ordered and the sequence does not matter) in Lottert between 1995 and 2020:")
print(rdd.take(10))

Top 10 most commonly drawn triple numbers (not ordered and the sequence does not matter) in Lottert between 1995 and 2020:
[((3, 8, 40), 8), ((12, 19, 47), 8), ((1, 21, 24), 8), ((23, 39, 47), 7), ((10, 38, 41), 7), ((1, 24, 42), 7), ((4, 9, 22), 7), ((18, 32, 44), 7), ((5, 38, 41), 7), ((7, 17, 20), 7)]
